In [1]:
%pip install -q -U bitsandbytes
%pip install -q -U git+https://github.com/huggingface/transformers.git
%pip install -q -U git+https://github.com/huggingface/peft.git

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [3]:
from peft import PeftModel, PeftConfig

peft_model_id = "Mistral-7B-Instruct_finetuned_on_1000_array_basics"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, quantization_config=bnb_config, return_dict=True, load_in_4bit=True, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
import numpy as np
import random

min_num_rows_and_columns = 1
max_num_rows_and_columns = 8

train_grid_names = [
    ["Train_Input_1", "Train_Output_1"],
    ["Train_Input_2", "Train_Output_2"],
    ["Train_Input_3", "Train_Output_3"],
    ["Train_Input_4", "Train_Output_4"],
    ["Train_Input_5", "Train_Output_5"],
]
test_grid_names = [
    ["Test_Input_1", "Test_Output_1"],
    ["Test_Input_2", "Test_Output_2"],
]

row_size = random.randint(min_num_rows_and_columns, max_num_rows_and_columns)
column_size = random.randint(min_num_rows_and_columns, max_num_rows_and_columns)
random_array = np.random.randint(0, 10, size=(row_size, column_size))

num_rows = len(random_array)
num_columns = len(random_array[0])

# Display grid in better visual format
print("\nVisualization of grid:\n")
grid = ""
for i in range(num_rows):
    for j in range(num_columns):
        grid += str(random_array[i][j])
    grid += "\n"
print(grid)

train_or_test = random.randint(0, 1)
if train_or_test == 0:
    grid_name = test_grid_names[random.randint(0, len(test_grid_names) - 1)][
        random.randint(0, len(test_grid_names[0]) - 1)
    ]
else:
    grid_name = train_grid_names[random.randint(0, len(train_grid_names) - 1)][
        random.randint(0, len(train_grid_names[0]) - 1)
    ]

random_row = random.randint(0, num_rows - 1)
random_column = random.randint(0, num_columns - 1)

instruction = "'" + grid_name + "' is an ARCSolver grid: "

instruction += "["
for i in range(num_rows):
    if i == 0:
        instruction += "["
    for j in range(num_columns):
        instruction += str(random_array[i][j])
        if j < num_columns - 1:
            instruction += ","
        else:
            instruction += "]"
    if i < num_rows - 1:
        instruction += ",["
    else:
        instruction += "]"

instruction += ". "

random_question = random.randint(0, 3)
if random_question == 0:
    # Row question
    instruction += "What is " + grid_name + "[" + str(random_row) + "]?"
    ground_truth = random_array[random_row]
elif random_question == 1:
    instruction += "How many rows does " + grid_name + " have?"
    ground_truth = num_rows
elif random_question == 1:
    instruction += "How many columns does " + grid_name + " have?"
    ground_truth = num_columns
else:
    # Square question
    instruction += (
        "What is "
        + grid_name
        + "["
        + str(random_row)
        + "]["
        + str(random_column)
        + "]?"
    )
    ground_truth = random_array[random_row][random_column]

query = instruction
prompt = '<s> [INST] ' + query + ' [/INST] '
print("Prompt:\n", prompt)
encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
model_inputs = encodeds.to("cuda")
generated_ids = model.generate(**model_inputs, max_new_tokens=256, do_sample=True, pad_token_id=tokenizer.eos_token_id)
decoded = tokenizer.batch_decode(generated_ids)
result = decoded[0]
print("\nResult:\n", result)

print("\nGround truth:\n", ground_truth)

# Tokenize the request text
tokenized_request = tokenizer.tokenize(result)

# Get the token length
token_length = len(tokenized_request)

print("\nNumber of tokens: ", token_length)


Visualization of grid:

4
1

Prompt:
 <s> [INST] 'Test_Output_1' is an ARCSolver grid: [[4],[1]]. What is Test_Output_1[1][0]? [/INST] 

Result:
 <s>  [INST] 'Test_Output_1' is an ARCSolver grid: [[4],[1]]. What is Test_Output_1[1][0]? [/INST] 1</s>

Ground truth:
 1

Number of tokens:  46
